# Exercise 7: Hate Speech Classification using Multinomial Naive Bayes

Instructions:
- You do not need to split your data. Use the training, validation and test sets provided below.
- Use Multinomial Naive Bayes to train a model that can classify if a sentence is a hate speech or non-hate speech
- A sentence with a label of zero (0) is classified as non-hate speech
- A sentence with a label of one (1) is classified as a hate speech

Apply text pre-processing techniques such as
- Converting to lowercase
- Stop word Removal
- Removal of digits, special characters
- Stemming or Lemmatization but not both
- Count Vectorizer or TF-IDF Vectorizer but not both

Evaluate your model by:
- Providing input by yourself
- Creating a Confusion Matrix
- Calculating the Accuracy, Precision, Recall and F1-Score

In [982]:
!pip install simplemma

In [983]:
import pandas as pd
import simplemma
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import wordnet
from sklearn.pipeline import make_pipeline


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [984]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import seaborn as sns
import re
import os, types

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, precision_score, recall_score, accuracy_score, balanced_accuracy_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [985]:
splits = {'train': 'unique_train_dataset.csv', 'validation': 'unique_validation_dataset.csv', 'test': 'unique_test_dataset.csv'}

**Training Set**

Use this to train your model

In [986]:
df_train = pd.read_csv("hf://datasets/mapsoriano/2016_2022_hate_speech_filipino/" + splits["train"])

**Validation Set**

Use this set to evaluate your model

In [987]:
df_validation = pd.read_csv("hf://datasets/mapsoriano/2016_2022_hate_speech_filipino/" + splits["validation"])

**Test Set**
  
Use this set to test your model

In [988]:
df_test = pd.read_csv("hf://datasets/mapsoriano/2016_2022_hate_speech_filipino/" + splits["test"])

## A. Understanding your training data

1. Check the first 10 rows of the training dataset

In [989]:
df_train.head(10)

,text,label
0,Presidential candidate Mar Roxas implies that ...,1
1,Parang may mali na sumunod ang patalastas ng N...,1
2,Bet ko. Pula Ang Kulay Ng Posas,1
3,[USERNAME] kakampink,0
4,Bakit parang tahimik ang mga PINK about Doc Wi...,1
5,"""Ang sinungaling sa umpisa ay sinungaling hang...",1
6,Leni Kiko,0
7,Nahiya si Binay sa Makati kaya dito na lang sa...,1
8,Another reminderHalalan,0
9,[USERNAME] Maybe because VP Leni Sen Kiko and ...,0


2. Check how many rows and columns are in the training dataset using `.info()`

In [990]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21773 entries, 0 to 21772
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    21773 non-null  object
 1   label   21773 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 340.3+ KB


3. Check for NaN values

In [991]:
df_train.isna().sum()

,0
text,0
label,0


4. Check for duplicate rows

In [992]:
df_train.duplicated().sum()

0

5. Check how many rows belong to each class

In [993]:
df_train['label'].value_counts()

,count
label,
1,10994
0,10779


## B. Text pre-processing

6. Remove duplicate rows

In [994]:
df_train = df_train.drop_duplicates()

7. Remove rows with NaN values

In [995]:
df_train = df_train.dropna()

8. Convert all text to lowercase

In [996]:
df_train['text'] = df_train['text'].str.lower()

9. Remove digits, URLS and special characters

In [997]:
def clean_text(text):
    return re.sub(r'[^A-Za-z\s]', '', text)

df_train['text'] = df_train['text'].apply(clean_text)

In [998]:
df_train.head(10)

,text,label
0,presidential candidate mar roxas implies that ...,1
1,parang may mali na sumunod ang patalastas ng n...,1
2,bet ko pula ang kulay ng posas,1
3,username kakampink,0
4,bakit parang tahimik ang mga pink about doc wi...,1
5,ang sinungaling sa umpisa ay sinungaling hangg...,1
6,leni kiko,0
7,nahiya si binay sa makati kaya dito na lang sa...,1
8,another reminderhalalan,0
9,username maybe because vp leni sen kiko and th...,0


10. Remove stop words

In [999]:
stop_words_e = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in text.split() if word.lower() not in stop_words])

df_train['text'] = df_train['text'].apply(remove_stopwords)

stop_words = [
    "ako", "sa", "akin", "ko", "aking", "sarili", "kami", "atin", "ang", "aming", "amin", "ating", "ka", "iyong", "iyo", "inyong",
    "siya", "kanya", "mismo", "ito", "nito", "kanyang", "sila", "nila", "kanila", "kanilang", "kung", "ano", "alin", "sino",
    "kanino", "na", "mga", "iyon", "am", "ay", "maging", "naging", "mayroon", "may", "nagkaroon", "pagkakaroon", "gumawa",
    "ginagawa", "ginawa", "paggawa", "ibig", "dapat", "maaari", "marapat", "kong", "ikaw", "tayo", "hindi", "namin", "gusto",
    "nais", "niyang", "nilang", "niya", "huwag", "ginawang", "gagawin", "maaaring", "sabihin", "narito", "kapag", "ni",
    "nasaan", "bakit", "paano", "kailangan", "walang", "katiyakan", "isang", "at", "pero", "o", "dahil", "bilang", "hanggang",
    "habang", "ng", "pamamagitan", "para", "tungkol", "laban", "pagitan", "panahon", "bago", "pagkatapos", "itaas", "ibaba",
    "mula", "pataas", "pababa", "palabas", "ibabaw", "ilalim", "muli", "pa", "minsan", "dito", "doon", "saan", "lahat",
    "anumang", "kapwa", "bawat", "ilan", "karamihan", "iba", "tulad", "lamang", "pareho", "kaya", "kaysa", "masyado",
    "napaka", "isa", "bababa", "kulang", "marami", "ngayon", "kailanman", "sabi", "nabanggit", "din", "kumuha", "pumunta",
    "pumupunta", "ilagay", "makita", "nakita", "katulad", "mahusay", "likod", "kahit", "paraan", "noon", "gayunman",
    "dalawa", "tatlo", "apat", "lima", "una", "pangalawa"]

# stop_words = [
#     "ang", "ng", "sa", "na", "at", "ay", "ko", "mo", "siya", "kami",
#     "tayo", "kayo", "nila", "ito", "iyon", "doon", "dito", "hindi",
#     "wala", "may", "mga"]

df_train['text'] = df_train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [1000]:
df_train.head(10)

,text,label
0,presidential candidate mar roxas implies that ...,1
1,parang mali sumunod patalastas nescaf coffee b...,1
2,bet pula kulay posas,1
3,username kakampink,0
4,parang tahimik pink about doc willie ong no re...,1
5,sinungaling umpisa sinungaling dulo vp leni ma...,1
6,leni kiko,0
7,nahiya si binay makati lang mandaluyong bwiset...,1
8,another reminderhalalan,0
9,username maybe because vp leni sen kiko and th...,0


11. Use Stemming or Lemmatization

In [1001]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized_tokens)

df_train['text'] = df_train['text'].apply(lemmatize_text)

## C. Training your model

12. Put all text training data in variable **X_train**

In [1002]:
from sklearn.model_selection import train_test_split

X_train = df_train['text']

13. Put all training data labels in variable **y_train**

In [1003]:
y_train = df_train['label']

14. Use `CountVectorizer()` or `TfidfVectorizer()` to convert text data to its numerical form.

Put the converted data to **X_train_transformed** variable

In [1004]:
vectorizer = CountVectorizer()
X_train_transformed = vectorizer.fit_transform(X_train)

15. Create an instance of `MultinomalNB()`

In [1005]:
model = MultinomialNB(alpha=1)

16. Train the model using `.fit()`

In [1006]:
model.fit(X_train_transformed, y_train)

MultinomialNB(alpha=1)

## D. Evaluate your model

17. Use `.predict()` to generate model predictions using the **validation dataset**


- Put all text validation data in **X_validation** variable

- Convert **X_validation** to its numerical form.

- Put the converted data to **X_validation_transformed**

- Put all predictions in **y_validation_pred** variable

In [1007]:
X_validation = df_validation['text']
X_validation_transformed = vectorizer.transform(X_validation)
y_validation_pred = model.predict(X_validation_transformed)

18. Get the Accuracy, Precision, Recall and F1-Score of the model using the **validation dataset**

- Put all validation data labels in **y_validation** variable

In [1008]:
y_validation = df_validation['label']

accuracy = accuracy_score(y_validation, y_validation_pred)
precision = precision_score(y_validation, y_validation_pred)
recall = recall_score(y_validation, y_validation_pred)
f1 = f1_score(y_validation, y_validation_pred)

print(f"Accuracy (Validation): {accuracy}")
print(f"Precision (Validation): {precision}")
print(f"Recall (Validation): {recall}")
print(f"F1 Score (Validation): {f1}")

Accuracy (Validation): 0.8289285714285715
Precision (Validation): 0.8070866141732284
Recall (Validation): 0.8692579505300353
F1 Score (Validation): 0.8370193943518204


19. Create a confusion matrix using the **validation dataset**

In [1009]:
from sklearn.metrics import confusion_matrix

cm_validation = confusion_matrix(y_validation, y_validation_pred)
print("Confusion Matrix (Validation):")
print(cm_validation)

Confusion Matrix (Validation):
[[1091  294]
 [ 185 1230]]


20. Use `.predict()` to generate the model predictions using the **test dataset**


- Put all text validation data in **X_test** variable

- Convert **X_test** to its numerical form.

- Put the converted data to **X_test_transformed**

- Put all predictions in **y_test_pred** variable

In [1010]:
X_test = df_test['text']

X_test_transformed = vectorizer.transform(X_test)

y_test_pred = model.predict(X_test_transformed)

21. Get the Accuracy, Precision, Recall and F1-Score of the model using the **test dataset**

- Put all test data labels in **y_validation** variable



In [1011]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_test = df_test['label']

accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)


print(f"Accuracy (Test): {accuracy_test}")
print(f"Precision (Test): {precision_test}")
print(f"Recall (Test): {recall_test}")
print(f"F1 Score (Test): {f1_test}")

Accuracy (Test): 0.8188612099644128
Precision (Test): 0.7941760423560555
Recall (Test): 0.8583690987124464
F1 Score (Test): 0.8250257820556892


22. Create a confusion matrix using the **test dataset**

In [1012]:
cm_test = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix (Test):")
print(cm_test)

Confusion Matrix (Test):
[[1101  311]
 [ 198 1200]]


## E. Test the model

23. Test the model by providing a non-hate speech input. The model should predict it as 0

In [1013]:
new_input = ["mahal kita"]
new_input_transformed = vectorizer.transform(new_input)
prediction = model.predict(new_input_transformed)

print("Prediction:", prediction)

predict = '[0]'

Prediction: [0]


24. Test the model by providing a hate speech input. The model should predict it as 1

In [1015]:
new_input = ["Tangina mo!"]
new_input_transformed = vectorizer.transform(new_input)
prediction = model.predict(new_input_transformed)

print("Prediction:", prediction)

Prediction: [1]
